## 패키지 설치

In [1]:
!pip install openai langchain tiktoken langchain_community langchain-openai gradio

## 라이브러리 임포트

In [16]:
import os

import pandas as pd
import numpy as np
from numpy import dot
from numpy.linalg import norm

import openai
from openai import OpenAI
from langchain.embeddings import OpenAIEmbeddings

import gradio as gr

In [3]:
os.environ['OPENAI_API_KEY'] =  "api_key"


## 임베딩

In [4]:
# 질문 등록
data = ['구글 드라이브 링크 알려줘',
        '패들렛 링크 알려줘',
        '코랩 링크 알려줘',
        '노션 링크 알려줘',
        'QR코드 링크 줘']

df = pd.DataFrame(data, columns=['question'])
df

,question
0,구글 드라이브 링크 알려줘
1,패들렛 링크 알려줘
2,코랩 링크 알려줘
3,노션 링크 알려줘
4,QR코드 링크 줘


In [5]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

<ipython-input-5-472716ae61dc>:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")


In [6]:
def get_embedding(text):
  return embeddings.embed_query(text)

In [7]:
# 질문들 임베딩
df['embedding'] = df.apply(lambda row: get_embedding(row.question), axis=1)

In [8]:
# 질문에 대한 답변 등록
df['response'] = [
    "구글 드라이브 링크는 https://drive.google.com/drive/folders/1BCnlhlNxuUAbREvZXHe_m6sJ5YjQ73Te 야. 여기서 강의자료와 코랩 링크를 얻을 수 있어.",
    "패들렛 링크는 https://padlet.com/aiffelcampus/llm-2-ztat1mdgu9l6wvy야. QR코드, 출석병결 관련 안내, 강의영상 링크 등의 정보를 얻을 수 있어.",
    "구글 드라이브 링크 https://drive.google.com/drive/folders/1BCnlhlNxuUAbREvZXHe_m6sJ5YjQ73Te 의 폴더에 들어가면 텍스트 파일에 코랩 링크 주소가 있어. 거기서 찾아봐.",
    "노션 링크는 https://www.notion.so/modulabs/_AI-LLM-_2-25-02-18-25-05-17-1975c8e5427d809b9ee4ff814f4117b4 야.",
    "QR 코드는 패들렛 링크에 들어가서 출결을 체크할 수 있어. https://padlet.com/aiffelcampus/llm-2-ztat1mdgu9l6wvy 이 링크로 들어가봐"
]

In [9]:
df

,question,embedding,response
0,구글 드라이브 링크 알려줘,"[-0.019984146497294433, -0.003371255923994703,...",구글 드라이브 링크는 https://drive.google.com/drive/fol...
1,패들렛 링크 알려줘,"[-0.012377640184230813, -0.0011916309548691997...",패들렛 링크는 https://padlet.com/aiffelcampus/llm-2-...
2,코랩 링크 알려줘,"[-0.005571689385430938, -0.008415468860804439,...",구글 드라이브 링크 https://drive.google.com/drive/fold...
3,노션 링크 알려줘,"[-0.011588773323398825, -0.018601088759480935,...",노션 링크는 https://www.notion.so/modulabs/_AI-LLM-...
4,QR코드 링크 줘,"[-0.01412576697677204, 0.0059893773075113494, ...",QR 코드는 패들렛 링크에 들어가서 출결을 체크할 수 있어. https://padl...


## 유사도 기반 응답 함수

In [10]:
def cos_sim(A, B):
  return dot(A, B)/(norm(A)*norm(B))

def return_answer_candidate(df, query, threshold=0.9): # 0.9 이상인것중에 제일 높은 유사도를 띄는 것 반환 0.9 미만은 반환 안함.
    # 입력된 query의 임베딩 생성
    query_embedding = get_embedding(query)

    # similarity 계산 및 데이터프레임에 추가
    df["similarity"] = df['embedding'].apply(lambda x: cos_sim(np.array(x), np.array(query_embedding)))

    # threshold 이상인 결과 필터링
    filtered_results = df[df["similarity"] >= threshold]

    # 조건에 맞는 결과가 있으면 가장 큰 similarity의 response 반환, 없으면 메시지 반환
    if not filtered_results.empty:
        best_match = filtered_results.loc[filtered_results["similarity"].idxmax()]  # similarity가 최대인 행 선택
        return best_match["response"]  # response 컬럼 값 반환
    else:
        return None  # 조건에 맞는 결과가 없을 때 메시지 반환

In [11]:
sim_result = return_answer_candidate(df, '구글 드라이브 링크 알려줄래?') # 질문은 "구글 드라이브 링크 알려줘"로 등록했는데 조금 다르게 질문해도 높은 유사도를 보임.

In [12]:
sim_result

'구글 드라이브 링크는 https://drive.google.com/drive/folders/1BCnlhlNxuUAbREvZXHe_m6sJ5YjQ73Te 야. 여기서 강의자료와 코랩 링크를 얻을 수 있어.'

## 유사도 기반으로 응답할수 없는 경우 일반 gpt 실행해서 응답줘라.

In [17]:
def general_answer_machine(user_query): # 0.9미만의 유사도일때는 그냥 일반 응답 주기위한 용도.
    # 입력 값이 비어 있는 경우 처리
    if not user_query or not user_query.strip():
        return "Error: The query is empty. Please provide a valid input."

    # 시스템 프롬프트 정의
    system_prompt = (
        "You are a multi-purpose assistant designed to help users solve problems efficiently. "
        "Provide step-by-step guidance or actionable advice as needed."
    )
    client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
    try:
        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_query}
            ],
            max_tokens=1000,
            temperature=0.1,
            top_p=0.9,
        )

        # 응답 반환
        return response.choices[0].message.content.strip()

    except Exception as e:
        # API 호출 실패 시 예외 처리
        return f"Error: Unable to process the request. Details: {str(e)}"


In [19]:
general_answer_machine(user_query='네이버 주소 뭐야?')

'네이버의 웹사이트 주소는 www.naver.com입니다. 웹 브라우저의 주소창에 입력하시면 네이버의 메인 페이지로 이동할 수 있습니다.'

## UI

In [20]:
# Gradio 인터페이스 정의
def chatbot_interface(user_query):
    answer = return_answer_candidate(df, user_query)
    if not answer:
        answer = general_answer_machine(user_query)
    return answer

with gr.Blocks() as app:
    gr.Markdown("## 유사도 기반 응답 시스템")

    with gr.Row():
        user_input = gr.Textbox(label="질문을 입력하세요", placeholder="예: 오늘날씨어때?")
        output = gr.Textbox(label="응답")

    submit_button = gr.Button("응답 요청")

    submit_button.click(chatbot_interface, inputs=user_input, outputs=output)

# 앱 실행
app.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://7ca088da4e94a8ca75.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
